In [ ]:
%load_ext autoreload
%autoreload 2
from visualize import (
    plot_players_with_numbers,
    visualize_frame,
    shot_frames_navigator,
    animate_pitch,
    label_shot_chains_with_options
)
from data import load_and_process_soccer_events, load_and_process_metadata
from label import pos_labeling
from IPython.display import HTML

In [ ]:
event_df, players_df = load_and_process_soccer_events("/home/soccerdata/FIFA_WorldCup_2022/Event Data")
metadata_df = load_and_process_metadata("/home/soccerdata/FIFA_WorldCup_2022/Metadata")

In [ ]:
pos_chains = pos_labeling(event_df, chain_len=6)

In [ ]:
anim = animate_pitch(pos_chains[10], event_df, players_df, metadata_df, interval=500)
HTML(anim.to_jshtml())

In [ ]:
accepted_chains = label_shot_chains_with_options(
    chains=pos_chains,                
    chain_range=(0, 3),  
    event_df=event_df,
    players_df=players_df,
    metadata_df=metadata_df,
    output_dir="/home/soccerdata/frames",
    show_video=True,
    interval=1000
)